In [1]:
import requests
import pandas as pd

from power.ml_ops.data import get_pv_data, clean_pv_data, get_data_with_cache
from power.ml_ops.model import model_yesterday
from power.ml_ops.registry import load_model
from datetime import datetime, timedelta

from pathlib import Path
from power.params import *

2024-03-09 21:57:13.081959: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-09 21:57:19.799968: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-09 21:57:19.866316: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-09 21:57:28.361359: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
model = load_model()

data_processed_cache_path = Path(LOCAL_DATA_PATH).joinpath("processed", f"processed_pv.csv")
query = f"""
    SELECT *
    FROM {GCP_PROJECT}.{BQ_DATASET}.processed_pv
    ORDER BY utc_time
"""
data_pv_clean = get_data_with_cache(
    gcp_project=GCP_PROJECT,
    query=query,
    cache_path=data_processed_cache_path,
    data_has_header=True
    )

input_date = "2012-09-10 00:00:00"


Load latest model from local registry...

Load latest model from disk...


2024-03-09 21:57:36.398418: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-03-09 21:57:36.406173: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-03-09 21:57:36.411023: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

✅ Model loaded from local disk

Load data from local CSV...
✅ Data loaded, with shape (376944, 3)


In [17]:
input_prediction_date = "2019-12-10 00:00:00"
# make api call
base_url = "http://127.0.0.1:8000"

params_model ={
    'input_date':input_prediction_date
    }

# model
# endpoint_model = "/predict"
# url_model = f"{base_url}{endpoint_model}"
# response_model = requests.get(url_model, params)

# baseline



# data
params_data ={
    'input_date':input_prediction_date,
    'n_days': 10
    }

endpoint_data = "/extract_data"
url_data = f"{base_url}{endpoint_data}"
response_data = requests.get(url_data, params_data).json()




# response_model = requests.get(url_model, params=params).json()
# model_df = pd.DataFrame(response_model)

# response_data = requests.get(url_data, params=params).json()
# data_df = pd.DataFrame(response_data)

response_data


{'2019-12-10 00:00:00': {'days_before': [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.003,
   0.032,
   0.069,
   0.088,
   0.073,
   0.047,
   0.013,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.004,
   0.047,
   0.098,
   0.115,
   0.111,
   0.086,
   0.049,
   0.011,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.02,
   0.039,
   0.047,
   0.053,
   0.038,
   0.017,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.004,
   0.038,
   0.06,
   0.091,
   0.075,
   0.081,
   0.021,
   0.004,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.012,
   0.073,
   0.182,
   0.305,
   0.32,
   0.427,
   0.318,
   0.115,
   

In [4]:
def predict_baseline_yesterday(input_date: str):
    data = data_pv_clean[data_pv_clean['utc_time'] < input_date][-24:]
    values = data.electricity.to_list()
    return {input_date: values}

predict_baseline_yesterday(input_date)

{'2012-09-10 00:00:00': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.062,
  0.239,
  0.415,
  0.559,
  0.647,
  0.699,
  0.705,
  0.654,
  0.564,
  0.422,
  0.243,
  0.073,
  0.001,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0]}

In [10]:
def extract_pv_data(input_date: str, n_days=10):
    """
    Returns the power data for the n_days before the input date
    Also returns the power data for the following day
    """
    n_rows = 24 * n_days
    days_before = data_pv_clean[data_pv_clean['utc_time'] < input_date] \
                                        ['electricity'][-n_rows:].to_list()
    day_after = data_pv_clean[data_pv_clean['utc_time'] >= input_date] \
                                        ['electricity'][:24].to_list()


    extracted_data = {
        'days_before':days_before,
        'day_after':day_after
        }

    return {input_date: extracted_data}

extract_pv_data(input_date)

{'2012-09-10 00:00:00': {'days_before': [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.007,
   0.028,
   0.057,
   0.091,
   0.132,
   0.221,
   0.119,
   0.118,
   0.125,
   0.129,
   0.086,
   0.034,
   0.003,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.002,
   0.072,
   0.257,
   0.455,
   0.603,
   0.701,
   0.753,
   0.755,
   0.714,
   0.603,
   0.464,
   0.273,
   0.095,
   0.007,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.002,
   0.056,
   0.182,
   0.346,
   0.5,
   0.595,
   0.636,
   0.623,
   0.561,
   0.455,
   0.311,
   0.146,
   0.038,
   0.002,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.052,
   0.221,
   0.422,
   0.58,
   0.683,
   0.732,
   0.719,
   0.653,
   0.559,
   0.42,
   0.171,
   0.037,
   0.003,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.001,
   0.046,
   0.12,
   0.23,
   0.465,
   0.678,


In [12]:
def predict(input_date: str, n_days=2):
    pv_data_clean = data_pv_clean
    X_pred = pv_data_clean[pv_data_clean['utc_time'] < input_date][-48:]

    return {'dataframe to predict': X_pred.electricity.to_list()}

predict(input_date)

{'dataframe to predict': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.02,
  0.08,
  0.154,
  0.272,
  0.446,
  0.567,
  0.434,
  0.345,
  0.344,
  0.285,
  0.206,
  0.058,
  0.002,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.062,
  0.239,
  0.415,
  0.559,
  0.647,
  0.699,
  0.705,
  0.654,
  0.564,
  0.422,
  0.243,
  0.073,
  0.001,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0]}